In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
import plotly.express as px

# 1. Chargement des données

X_full = pd.read_parquet("dataset_full.parquet") 

# 2. Chargement du modèle
loaded_model = xgb.Booster()
loaded_model.load_model("xgboost_v3.json")

print(f"✅ Modèle et {len(X_full)} entreprises chargés.")

In [ ]:
# --- ÉTAPE A : PRÉPARATION & PRÉDICTION ---

# 1. Copie pour ne pas modifier l'original
X_input = X_full.copy()

# 2. Feature Engineering (Recréation des variables du modèle)
if "Economie sociale et solidaire unité légale" in X_input.columns:
    X_input['is_ess'] = X_input["Economie sociale et solidaire unité légale"].map({'O': 1, 'N': 0}).fillna(0)

# 3. One-Hot Encoding des secteurs
if 'libelle_section_ape' in X_input.columns:
    X_input = pd.get_dummies(X_input, columns=['libelle_section_ape'], prefix='APE')

# 4. Alignement strict avec les attentes du modèle (Colonnes manquantes et ordre)
for col in loaded_model.feature_names:
    if col not in X_input.columns:
        X_input[col] = 0

X_input = X_input[loaded_model.feature_names]

# 5. Calcul des prédictions brutes
dmatrix_full = xgb.DMatrix(X_input)
raw_preds = loaded_model.predict(dmatrix_full)

# --- ÉTAPE B : SCORING ET ANALYSE MÉTIER ---

# 1. Calcul de l'Indice de Risque Global (0-100)
risk_scores = pd.Series(raw_preds).rank(pct=True, ascending=False) * 100

# 2. Création du DataFrame de résultats consolidé
df_resultats = pd.DataFrame({
    'SIREN': X_full["SIREN"],
    'Dénomination': X_full["Dénomination de l'unité légale"],
    'Indice_Risque': risk_scores.values
}, index=X_full.index)

# 3. Fonction de calcul des probabilités par horizon avec nettoyage
def calculer_probabilites_propres(score):

    p_1an = 1 / (1 + np.exp(-(score - 85) / 3))
    p_2ans = 1 / (1 + np.exp(-(score - 70) / 5))
    p_3ans = 1 / (1 + np.exp(-(score - 55) / 7))
    
    probas = []
    for p in [p_1an, p_2ans, p_3ans]:
        val = round(p * 100, 2)
        probas.append(val if val >= 0.01 else 0.0)
    return probas[0], probas[1], probas[2]

# 4. Application des calculs d'horizons
df_resultats['Prob_1an'], df_resultats['Prob_2ans'], df_resultats['Prob_3ans'] = zip(*df_resultats['Indice_Risque'].map(calculer_probabilites_propres))

# 5. Attribution du Statut Expert
df_resultats['Statut_Expert'] = pd.cut(
    df_resultats['Indice_Risque'], 
    bins=[0, 55, 70, 85, 100], 
    labels=['🟢 SAIN', '🟡 OBSERVATION', '🟠 VIGILANCE', '🔴 CRITIQUE'],
    include_lowest=True
)

# 6. Affichage des résultats prioritaires
print(f"✅ Analyse terminée sur {len(df_resultats)} entreprises.")
cols_view = ['SIREN', 'Dénomination', 'Indice_Risque', 'Prob_1an', 'Prob_2ans', 'Prob_3ans', 'Statut_Expert']
display(df_resultats[cols_view].sort_values('Indice_Risque', ascending=False).head(15))

In [ ]:

# 6. Affichage final (SANS DOUBLONS)
print("✅ Diagnostic de survie consolidé terminé (Chiffres nettoyés) !")
cols_view = ['SIREN', 'Dénomination', 'Indice_Risque', 'Prob_1an', 'Prob_2ans', 'Prob_3ans', 'Statut_Expert']

# On ajoute .drop_duplicates sur le SIREN pour ne garder qu'une ligne par entreprise
df_propre = df_resultats[cols_view].drop_duplicates(subset=['SIREN'], keep='first')

# Affichage des 10 moins risqués (puisque tu fais tail sur un tri descendant)
display(df_propre.sort_values('Indice_Risque', ascending=False).tail(10))


In [ ]:
df_propre.shape